# Finite Zeal
**Description**: Words...

## Imports

In [1]:
import os
import re
import json
import math
import time
import boto3

from bedrock_sdk.prompt_template import PromptTemplate
from bedrock_sdk.prompt_template import FewShotTemplate

from bedrock_sdk.bedrock_client import ModelConfig
from bedrock_sdk.bedrock_client import BedrockClient

## Methods

## Configuration

In [2]:
bedrock = BedrockClient()

In [3]:
model_top = ModelConfig(
    model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=4096,
    temperature=0.01
)

model_upper = ModelConfig(
    model_id="us.anthropic.claude-3-5-sonnet-20240620-v1:0",
    max_tokens=4096,
    temperature=0.01
)

model_mid = ModelConfig(
    model_id="us.amazon.nova-pro-v1:0",
    max_tokens=4096,
    temperature=0.01
)

model_low = ModelConfig(
    model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    max_tokens=4096,
    temperature=0.01
)

model_exp = ModelConfig(
    model_id="us.meta.llama3-3-70b-instruct-v1:0",
    max_tokens=2048,
    temperature=0.01
)

## Prompts

### Prompt - 3-arc

In [4]:
prompt_3act_text = '''
<<system>>
You are an AI novelist helping authors write creative and engrossing novels.
Your current task is to shape the theme and overall acts of the novel.
You will be given an incomplete story idea from a user and need to write a complete and engrossing 3-act narrative.

Specifically, answer the following:
- Who is the the hero/heroes, and who might oppose them
- What are these people trying to do
- What the setting is, both geographically and socially
- What might complicate the drama
- Why will this be a long story and not a short one
- What kicks off the situation
- Where it will end
- What the genre is

Keep in mind (story):
A story is usually someone trying to do something, or prevent something.
The story ends when they've done it, or done something else that puts an end to the journey.
To spice things up they have obstacles, or conflict, which make it all more troublesome than they imagined when they started.

Keep in mind (characters):
All good fiction comes down to characters, no matter what the genre.
Thrillers need active characters, but so does literary fiction.

Keep in mind (level):
Generate a sophisticated literary fiction outline targeting educated adult readers.
Include complex psychological themes, morally ambiguous characters, and subtle societal commentary.
Avoid simplistic resolutions and YA tropes.

The 3-act structure should include a detailed description of:
1. Setup (introduces the protagonist(s) and inciting incident)
2. Confrontation (rising action, tension, and challenges)
3. Resolution (final conflict and conclusion)

<<user>>
Story Prompt:
{{STORY_PROMPT}}
======================================================
Read the above story prompt and write a 3-act narrative structure around it.
Write the narrative as a markdown document starting with the tag <markdown> and ending with </markdown>.

<<assistant>>
{{RESPONSE_START_3ACT}}
'''

In [5]:
prompt_3act = PromptTemplate(
    name='prompt-ainfnitejest-3act',
    description='TBA',
    content=prompt_3act_text
)

### Prompt - 3-arc follow-up

In [6]:
prompt_3actu_text = '''
<<system>>
You are an AI novelist helping authors write creative and engrossing novels.
Your current task is to shape a 3-arc narrative by adding compelling and creative plotlines. 
You will be given a story narrative idea from a user and need to re-write it as a complete and engrossing 3-act narrative.

Specifically:
- Use at least one (sometimes more) plot situations

Plot Situations:
- abduction
- adultery
- all sacrificed for passion
- ambition
- an enemy loved 
- conflict with a god
- crime of love
- crime pursued by vengeance
- daring enterprise
- deliverance
- disaster
- discovery of the dishonor of a loved one
- enmity of a kinsmen
- erroneous judgement
- falling prey to cruel misfortune
- fatal imprudence
- involuntary crimes of love
- loss of a loved one
- madness
- mistaken jealousy
- murderous adultery
- necessity of sacrificing loved ones 
- obstacles
- obtaining
- pursuit
- recovery of a lost one
- remorse
- revolt
- rivalry of kinsmen
- rivalry of supreme and inferior
- self-sacrificing for an ideal
- self-sacrificing for a kindred
- slaying of a kinsmen unrecognized
- supplication
- the enigma
- vengeance taken for kindred upon kindred

<<user>>
Story Acts:
{{STORY_ACTS}}
======================================================
Corrective Notes:
{{CORRECTIVE_NOTES}}
======================================================
From the 3-act narrative and corrective notes (if any), rewrite adding any plot lines that seem needed to enhance the quality of the novel.
Write the narrative as a markdown document starting with the tag <markdown> and ending with </markdown>.

<<assistant>>
{{RESPONSE_START_3ACTU}}
'''

In [7]:
prompt_3actu = PromptTemplate(
    name='prompt-ainfnitejest-3actu',
    description='TBA',
    content=prompt_3actu_text
)

### Prompt - Characters

In [8]:
prompt_characters_text = '''
<<system>>
You are an AI novelist helping authors write creative and engrossing novels.
Your current task is to shape the characters of the novel. 
You will be given a story narrative from a user and must write detailed character profiles with psychological depth.

# Remember:
- Characters are the heart of your story. Developing their personal journeys early ensures they feel real and relatable.
- By defining how each character grows and changes, you create consistency in their actions, making emotional beats more impactful throughout the narrative.
- Intertwining character arcs from the start builds more dynamic relationships and can add layers of tension, conflict, and resolution.
- You are not bound to only write profiles for the characters written in the provided narrative, but can add new minor characters, and slightly modify the main characters.

# Keep in mind (level):
Generate a sophisticated literary fiction outline targeting educated adult readers.
Include complex psychological themes, morally ambiguous characters, and subtle societal commentary.
Avoid simplistic resolutions and YA tropes.

## Steps to Follow:

### Define Each Character’s Core Identity:
Start by understanding who your characters are. What are their core beliefs, values, goals, and fears?
This will serve as a baseline to show how they change throughout the story.

### Establish Personal Goals and Motivations:
Give each character a clear goal or desire that drives their actions. What do they want to achieve? What is standing in their way?

### Map Out Their Growth:
Decide how each character will evolve over the course of the story. What internal or external experiences will lead to their growth or transformation?
Will they overcome flaws, make sacrifices, or gain new insights?

### Determine Relationships and Conflicts:
Identify how characters' goals and values interact with one another. Do they align or clash? How do their relationships add tension or drive the plot forward?
Consider how these relationships evolve alongside their personal arcs.

### Visualize Intersecting Arcs:
Plot the key moments where character arcs intersect. Are there moments where one character's growth affects another's? How do these connections build emotional stakes?

### Consistency Across the Story:
Revisit these arcs as you draft the story, ensuring the characters’ actions align with their development.
Check that their relationships deepen or shift in ways that reflect their changing identities and motivations.

### Tie Arcs to Emotional Beats:
Use major emotional moments in the story (victories, losses, betrayals) to further your characters' development.
By mapping these arcs upfront, you can guide these beats toward satisfying character payoffs.

<<user>>
Story Acts:
{{STORY_ACTS}}
======================================================
From the story acts, write a full and diverse list of characters for the novel.
Write the narrative as a markdown document starting with the tag <markdown> and ending with </markdown>.

<<assistant>>
{{RESPONSE_START_CHARACTERS}}
'''

In [9]:
prompt_characters = PromptTemplate(
    name='prompt-ainfnitejest-characters',
    description='TBA',
    content=prompt_characters_text
)

### Prompt - Characters follow-up

### Prompt - Chapters

In [10]:
prompt_chapters_text = '''
<<system>>
You are an AI novelist helping authors write creative and engrossing novels.
Your current task is to write the chapter arcs of the novel. 
You will be given a story narrative and character profile and must write chapter arcs following the narrative and psychological motivations and personalities of the characters.

Specifically:
- Each chapter should serve a distinct purpose in driving the overall story forward. Developing chapter arcs ensures that every scene contributes to the plot, builds tension, and deepens character development.
- By designing chapter arcs, you give each chapter its own mini-journey, making the narrative more engaging and providing readers with satisfying payoffs along the way.

## Steps to Follow:

### Start with the Purpose of the Chapter:
- Identify the main goal of the chapter. Is it to advance the plot, reveal character motivations, deepen a relationship, or introduce a conflict?
- Every chapter should have a clear objective that moves the story forward.

### Establish the Conflict or Tension:
- Introduce a challenge or conflict that the characters must deal with in the chapter. This creates tension and keeps the reader invested.
- The conflict can be external (an event or situation) or internal (a character's personal struggle).

### Set Up an Arc Within the Chapter:
- Like a mini-story, your chapter should have a beginning, middle, and end.
- Beginning: Set the scene and introduce the key players and problem.
- Middle: Escalate the tension or conflict.
- End: Provide some form of resolution or a cliffhanger that leads into the next chapter.

### Link the Chapter Arc to the Broader Story:
- Ensure that the chapter's events tie into the larger narrative. How does this chapter connect to the main plot or the character arcs you’ve already established?
- Think of the chapter as a stepping stone. What does it add to the overall story?

### Build Emotional and Narrative Payoffs:
- Each chapter should build toward an emotional or narrative payoff.
- This could be a small victory, a shocking revelation, a deepened relationship, or a turning point in the character’s journey.

### Create Momentum for the Next Chapter:
- End your chapter in a way that makes the reader want to continue. You can do this with a cliffhanger, a new mystery, or a shift in the character’s mindset or circumstances.
- Make sure the next chapter is set up smoothly so the momentum doesn’t stall.

### Balance Pacing:
- Consider the pacing of each chapter arc. Some chapters might be action-packed and fast-paced, while others can be quieter and more introspective.
- Varying the pacing keeps the reader engaged and allows for emotional highs and lows.
- Planning your chapters based on the length of your novel ensures consistent pacing, avoids filler content, and helps you stay focused on the key plot and character developments.

### Temporal Requirements:
- Ensure that temporal requirements are respected and acknowledged. It should be clear how much time has passed between scenes. This includes seasons, years, hours, etc.

### Example:
```
# Title: "[[Name]]"

## Act 1: "[[Name]]"

### Chapter 1: "[[Name]]" ([[number of scenes]])
- [[scene-1]] ([[number of pages]])
- [[scene-2]] ([[number of pages]])
.
.
.
- [[scene-n]] ([[number of pages]]) 
```

<<user>>
Story Acts:
{{STORY_ACTS}}
======================================================
Characters:
{{CHARACTERS}}
======================================================
From the story acts and characters, write a full list of chapter arcs.
Plan for the novel to be roughly {{N_PAGES}} pages long.
DO NOT use many more pages than the rough length desired.
Write the narrative as a markdown document starting with the tag <markdown> and ending with </markdown>.
Each chapter should list the number of scenes in the chapter.
And for each scene there should be an estimated number of pages for teh scene.

<<assistant>>
{{RESPONSE_START_CHAPTERS}}
'''

In [11]:
prompt_chapters = PromptTemplate(
    name='prompt-ainfnitejest-chapters',
    description='TBA',
    content=prompt_chapters_text
)

### Prompt - Chapters follow-up

### Prompt - Background

In [12]:
prompt_background_text = '''
<<system>>
You are an AI novelist helping authors write creative and engrossing novels.
Your current task is to write the chapter arcs of the novel. 
You will be given a story narrative and character profile and chapter arcs.
Use that to create background and world-building documents/material for the novel.

## Description:
A well-developed world provides the foundation for your story, giving depth to your characters, conflicts, and settings.
Writing background documents allows you to track details that don’t make it into the narrative but are crucial for maintaining internal consistency, immersion, and believability.
These documents also help clarify your world’s rules, history, and cultures, ensuring that the world feels lived-in and coherent.

## Steps to Follow:
### Start with World Structure:
- Geography: Map out the key locations, landmarks, countries, or regions in your world. Detail their climate, topography, and major cities or settlements.
- Political Systems: Identify how power is structured. Are there monarchies, republics, empires, or tribal societies? How do these political systems interact or come into conflict?
- Economy: What drives the economy in your world? Do different regions trade specific goods? Is there poverty, wealth disparity, or a focus on particular resources (e.g., mining, agriculture)?

### Create Historical Timelines:
- Major Historical Events: Write a timeline of significant events in your world’s history (wars, revolutions, discoveries, cultural shifts). How have these shaped the present day?
- Mythology and Legends: Develop key myths, legends, or religious beliefs that influence the worldview of the characters. Even if not directly included, they give a sense of tradition and depth.
- Cultural Shifts: Note any technological advancements, social movements, or influential figures that have impacted the development of civilizations or regions.

### Develop Societies and Cultures:
- Cultural Practices and Traditions: Define the cultural norms, festivals, rituals, and everyday customs for the different societies in your world. How do they differ from one another?
- Social Hierarchies: Determine how your world structures its social classes. Are there nobles, merchants, peasants? Is mobility between classes possible? What’s considered taboo or honorable?
- Religions and Belief Systems: Outline any dominant religions or spiritual beliefs. How do they shape laws, morality, and character behavior?
- Languages and Dialects: Even if you don’t create full languages, note the existence of different languages or dialects that distinguish regions or social classes.

### Establish Rules for Magic, Technology, or Supernatural Elements:
- Magic System (if applicable): Clearly define the rules of magic. What are the limits? Who can use it? What are the costs or dangers associated with magic use? Consistent magic rules avoid plot holes and build believability.
- Technology Level: Determine the level of technology present in your world. Are they in a medieval period, an industrial era, or a future with advanced tech? How does technology affect warfare, communication, and daily life?
- Supernatural Beings/Forces: If your world includes supernatural beings (gods, spirits, monsters), establish their origins, motives, and roles. How do they interact with humans? What is their influence on the world?

### Write Character Backgrounds and Lineages:
- Character Histories: Create documents for your key characters’ personal backgrounds. Include details about their upbringing, family, past relationships, and significant life events that won’t all be in the novel but influence their behavior and choices.
- Lineages or Dynasties: If your story involves noble houses or family lineages, map out family trees, important ancestors, and inheritances. Keep track of inter-family conflicts, alliances, and legacies.

### Document the Ecosystem:
- Flora and Fauna: Describe the plant life, animals, and ecosystems unique to your world. Are there dangerous creatures, medicinal plants, or mystical forests? These details enrich your descriptions and can serve as plot devices or obstacles for your characters.
- Environmental Influences: Consider how the environment shapes the societies living within it. Do harsh winters, deserts, or jungles influence cultural practices or survival strategies?

### Map Out the Economy and Trade:
- Resource Distribution: Identify key resources or commodities (e.g., metals, spices, magical substances) and where they’re found. How does the distribution of these resources affect trade, wealth, and power dynamics?
- Trade Routes and Conflicts: Determine the major trade routes and trade relationships between regions. Who benefits from these routes? Are there conflicts over resources or competition for control?

### Build a Reference Document for Tone and Style:
- Tone of the World: Decide on the overall tone of your world—grim and gritty, light and whimsical, or somewhere in between. This will help you maintain consistency in your descriptions and dialogue.
- Technology or Magic Terminology: Create a glossary for specific terms related to technology, magic, or other specialized elements of your world. This will help you use consistent language throughout the novel.

### Create Visual Aids or Maps:
- Maps: Draw basic maps of your world, even if only for your reference. Visualizing the layout of cities, kingdoms, and terrain helps you write more coherently about distances, travel, and setting.
- Diagrams: If your world involves complex machinery, architecture, or other intricate systems, sketch diagrams to keep your descriptions clear and consistent.

<<user>>
Story Acts:
{{STORY_ACTS}}
======================================================
Characters:
{{CHARACTERS}}
======================================================
Chapter Arcs:
{{CHAPTER_ARCS}}
======================================================
From the story acts, characters, and chapter arcs, write a full set of background and world-building material.
Write the documents as markdown documents starting with the tag <markdown> and ending with </markdown>.

<<assistant>>
{{RESPONSE_START_BACKGROUND}}
'''

In [13]:
prompt_background = PromptTemplate(
    name='prompt-ainfnitejest-background',
    description='TBA',
    content=prompt_background_text
)

### Prompt - Chapter Scene Generation

In [14]:
prompt_chapter_scene_text = '''
<<system>>
You are an AI novelist helping authors break down chapters into compelling scenes.
Your task is to create a detailed scene-by-scene breakdown for a specific chapter, ensuring each scene serves a purpose and maintains reader engagement.

Specifically:
- Each scene should have a clear purpose and advance either the plot, character development, or both
- Scenes should flow naturally from one to the next
- Each scene should have clear emotional stakes and tension
- The pacing and length of scenes should be balanced appropriately

## Scene Development Guidelines:

### Essential Scene Components:
- Setting & Atmosphere
- Point of View character
- Goal/Conflict/Stakes
- Key character interactions
- Emotional beats
- Plot advancement points

### Scene Structure:
- Entry point (hook)
- Rising action/complications
- Climactic moment or revelation
- Resolution or transition

### Technical Considerations:
- Approximate word count per scene
- POV consistency
- Time of day/location
- Characters present
- Key props or elements needed

### Scene-to-Scene Flow:
- Logical progression between scenes
- Maintaining tension across scene transitions
- Building toward chapter climax
- Connection to overall story arc

### Example
<chapter-1>
- scene 1: [[paragraph on what should happen in this scene]]
- scene 2: ...
.
.
.
- scene n: ...
</chapter-1>

<<user>>
Story Acts:
{{STORY_ACTS}}
======================================================
Characters:
{{CHARACTERS}}
======================================================
Background:
{{BACKGROUND}}
======================================================
Chapter Arcs:
{{CHAPTER_ARCS}}
======================================================
Novel so far:
{{NOVEL}}
======================================================
From the story acts, characters, background, chapter arcs, and break down the current chapter ({{CURRENT_CHAPTER}}) into individual scenes.
Write the scene breakdown as a markdown document list starting with the tag <chapter-{{CURRENT_CHAPTER}}> and ending with </chapter-{{CURRENT_CHAPTER}}>.
For the chapter, give a list of each scene, the number of pages and a paragraph of what will be in the scene.

<<assistant>>
{{RESPONSE_START_CHAPTER_SCENES}}
'''

In [15]:
prompt_chapter_scene = PromptTemplate(
    name='prompt-ainfnitejest-chapter-scene',
    description='TBA',
    content=prompt_chapter_scene_text
)

### Prompt - Scene Generation

#### Prompt Scene Recommendation

In [16]:
prompt_recommendations_text = '''
<<system>>
You are an AI novelist helping authors write creative and engrossing novels.
Your current task is to provide necessary infomration to the user to write the next scene of the novel. 
You will be given a story narrative, character profiles, background documents, and chapter arcs.
Use that to create a recommendation document to pass to the user for the next scene (several pages).

Specifically:
- Writing the Summary of What Has Happened So Far:
- Recommendation for the Rest of the Chapter:
- Recommendation for What Should Happen on This Page:

## Writing the Summary of What Has Happened So Far:
- Keep It Concise: Summaries should be brief (3–5 sentences) and focus on the key events and developments leading up to the current point. Highlight important plot points, character decisions, and emotional beats.
- Focus on Relevance: Only include details that are essential to understanding the current scene or chapter. Avoid going off on tangents or recapping minor events unless they directly impact what’s happening now.
- Track Character Arcs: Mention how the main characters have evolved. How have recent events affected their goals, motivations, or relationships?
- Maintain Chronological Clarity: Ensure the summary follows a logical sequence. If the story has multiple plotlines, be clear on which one is relevant at this moment.
### Example:
“So far, [Character A] has discovered a hidden secret in their past and is now on a quest to find the truth. Along the way, they’ve encountered [Character B], who is both a potential ally and a rival. Tensions have risen as [Character B] reveals their own motivations, conflicting with [Character A]’s. Both are now headed toward [Location], knowing that the next step in their journey may force them into direct confrontation.”

## Recommendation for the Rest of the Chapter:
- Highlight Key Objectives: List the primary goals for the remainder of the chapter (e.g., resolving a conflict, revealing new information, escalating tension).
- Set Emotional Beats: Identify key emotional shifts that need to happen (e.g., building trust, feeling betrayal, or discovering hope). Note how these should impact the characters' relationships or development.
- Guide the Plot Forward: Point out how the chapter should move the plot closer to the next major event or turning point. Should there be a reveal? A new conflict? A twist?
- Mention Necessary Transitions: If this chapter needs to set up the next one, suggest a smooth transition. What information should be revealed now to make the next chapter feel logical and connected?
- Balance Action and Reflection: Offer guidance on pacing—does the rest of the chapter need more action, or should there be moments of introspection to slow things down and develop the characters?
### Example:
“For the rest of the chapter, focus on the rising tension between [Character A] and [Character B]. Their conversation should lead to a subtle reveal that deepens their rivalry but also hints at a shared history they weren’t aware of. By the end of the chapter, set up a confrontation at [Location], leaving the reader anticipating the fallout.”

## Recommendation for What Should Happen on This Page:
- Be Specific: Focus on what is meant to happen on the page in terms of dialogue, actions, or descriptions. Mention the main event that needs to unfold in that space (e.g., a conversation, an action sequence, an emotional beat).
- Guide Tone and Atmosphere: Suggest how the mood should feel on the page (e.g., tense, light, reflective), and provide notes on any imagery, metaphors, or sensory details that will enhance the scene.
- Character Focus: State how the characters should behave or react, considering their current emotional state and goals. Is one character growing more suspicious, or is there a moment of vulnerability?
- Pace the Page: Consider pacing—how much information or action should be introduced? If it’s a slow moment, suggest ways to stretch out descriptions or inner thoughts. If it’s fast-paced, focus on brevity and movement.
- Include a Transition Element: Recommend how to end the page in a way that leads into the next one smoothly. Should there be a lingering question, a cliffhanger, or a moment of calm?
### Example:
“In this scene, focus on [Character A] starting to question [Character B]’s motives. They exchange loaded dialogue, with [Character A] growing more suspicious with every evasive answer. The tone should be tense, with subtle hints that both characters are preparing for something more dramatic. End the scene with [Character A] deciding whether to confront [Character B] directly or let the moment pass.”

## General Rules to Follow:
- Clarity Over Complexity: Make sure each summary and recommendation is clear and actionable, so it’s easy to follow when drafting the page.
- Consistency with Arcs: Always align the recommendations with the character and chapter arcs you’ve planned out.
- Stay Flexible: While following this structure, leave room for inspiration or unexpected changes in direction as you write.
- Keep the Big Picture in Mind: Even on a page-by-page level, think about how each scene or moment contributes to the overall narrative and character development.

<<user>>
Story Acts:
{{STORY_ACTS}}
======================================================
Characters:
{{CHARACTERS}}
======================================================
Background:
{{BACKGROUND}}
======================================================
Chapter Arcs:
{{CHAPTER_ARCS}}
======================================================
Novel so far:
{{NOVEL}}
======================================================
From the story acts, characters, background, chapter arcs, and previous pages give summaries and recommendations on what I should write for this scene (Scene: {{SCENE_NO}}) and current chapter (Current Chapter: {{CURRENT_CHAPTER}}).
Write the summary and recommendations as a text document for the scene starting with the tag <markdown> and ending with </markdown>.

<<assistant>>
{{RESPONSE_START_RECOMMENDATIONS}}
'''

In [17]:
prompt_recommendations = PromptTemplate(
    name='prompt-ainfnitejest-recommendations',
    description='TBA',
    content=prompt_recommendations_text
)

#### Prompt Scene Generation

In [18]:
prompt_scene_text = '''
<<system>>
You are an AI novelist helping authors write creative and engrossing novels.
Your current task is to write the next scene of the novel. 
You will be given a story narrative, character profiles, background documents, chapter arcs, and previous pages.
Use that to create a recommendation document to pass to the user for the next page.

Specifically:
- Following the notes provided by the user, you will write a scene for the novel.

Keep in mind (level):
- Generate a sophisticated literary fiction outline targeting educated adult readers.
- Include complex psychological themes, morally ambiguous characters, and subtle societal commentary.
- Avoid simplistic resolutions and YA tropes.

## PACING AND PLOT CONTROL GUIDELINES

1. STRICT ADHERENCE TO CURRENT MOMENT
- Focus ONLY on the immediate scene's events as outlined in the chapter arc
- Do not reference or hint at future events unless specifically indicated in the current scene outline
- Avoid any "little did they know" or similar future-pointing phrases
- Stay within the current character's knowledge at this specific point in time

2. SCENE BOUNDARIES
- Each page/scene should only cover events specified for that specific story beat
- If multiple plot points are listed for a chapter, only reveal the ones designated for the current scene
- Do not rush to get to later plot points or revelations
- Allow scenes to breathe and develop naturally within their allocated space

3. REVELATION CONTROL
- Only reveal information that has been explicitly marked for this scene
- Maintain mystery elements until their designated reveal points
- Do not provide hints or foreshadowing unless specifically instructed
- Keep character knowledge consistent with their current point in the story

4. TEMPORAL ANCHORING
- Ground the scene firmly in its present moment
- Focus on immediate sensory details and current character thoughts
- Avoid forward-looking statements or predictions
- Build tension through immediate scene elements rather than future hints

5. PLOT POINT PACING
For each scene/page, strictly observe:
- ONLY include plot points marked for this specific scene
- DO NOT advance plot elements marked for later scenes
- DO NOT include foreshadowing unless explicitly instructed
- DO NOT rush through current scene elements to get to later points

6. DEVELOPMENT GUIDELINES
When writing the scene:
- Fully develop the designated moment
- Include appropriate detail and sensory information
- Allow character reactions and emotions to unfold naturally
- Take time with important character interactions

7. SPECIFIC RESTRICTIONS
DO NOT:
- Jump ahead in the story
- Include future plot points
- Add unplanned foreshadowing
- Rush through designated scene elements
- Provide hints about future developments
- Skip over detailed character reactions
- Compress multiple plot points into one scene

8. CURRENT SCENE FOCUS
Concentrate on:
- Immediate action and dialogue
- Present character emotions and thoughts
- Current scene setting and atmosphere
- Designated plot points for this specific moment

9. PACING METRICS
For each scene/page:
- Confirm scene matches designated word count/length
- Ensure proper development of current plot points
- Verify no future elements are included
- Check that pacing matches scene's intended tone

10. QUALITY CHECKS
Before completing the scene, verify:
- Only current plot points are included
- No unintended future references exist
- Scene is fully developed at appropriate pace
- Character knowledge stays consistent with current point
- Designated revelations occur only when planned

## REMINDER
- The goal is to create fully realized scenes that live in their present moment, allowing the story to unfold at its intended pace without rushing or previewing future developments.
- A scene will consist of multiple pages of text.
- Follow the notes provided, but try to remain accurate to the story path.
- Global notes follow the novel throughout the novel or chapter arc.
- Local notes are only applicable to the current scene.

<<user>>
Story Acts:
{{STORY_ACTS}}
======================================================
Characters:
{{CHARACTERS}}
======================================================
Background:
{{BACKGROUND}}
======================================================
Chapter Arcs:
{{CHAPTER_ARCS}}
======================================================
Novel so far:
{{NOVEL}}
======================================================
Global Notes:
{{GLOBAL_NOTES}}
======================================================
Chapter Notes:
{{CHAPTER_NOTES}}
======================================================
Scene Notes:
{{SCENE_NOTES}}
======================================================
From the story acts, characters, background, chapter arcs, previous scenes, and page notes, write the next scene in the novel.
Use the details in the 
Remember this novel is intended to be around {{N_PAGES}}.
Write the scene as text pages starting with the tag <scene-{{SCENE_NO}}> and ending with </scene-{{SCENE_NO}}>.
ALWAYS write the number of pages the scene will cover.

<<assistant>>
{{RESPONSE_START_SCENE}}
'''

In [19]:
prompt_scene = PromptTemplate(
    name='prompt-ainfnitejest-scene',
    description='TBA',
    content=prompt_scene_text
)

In [20]:
### Prompt - Decisions

In [21]:
#### Prompt end of chapter

In [22]:
#### Prompt end of book

### Prompt - Critics

#### Prompt Continuity Critic

In [23]:
prompt_continuity_text = '''
<<system>>
You are an AI continuity critic helping authors maintain consistency and logical flow in their novels.
Your task is to analyze the current novel content, background materials, and character profiles to identify any continuity issues and provide recommendations for maintaining narrative coherence.

Specifically review and provide:
- Continuity Analysis of Recent Events
- Timeline Verification
- Character Consistency Check
- World-Building Consistency
- Recommendations for Maintaining Continuity

## Continuity Analysis of Recent Events:
- Track Plot Threads: Review recent events and ensure they follow logically from previous developments
- Check Time Flow: Verify events happen in a reasonable timeframe and sequence
- Location Tracking: Ensure character movements and locations make sense
- Cause and Effect: Confirm that story events have proper setup and consequences
### Example:
"In the last three chapters, Character A traveled from City X to City Y, which normally takes two days, but the narrative suggests only hours passed. This creates a timing inconsistency that needs addressing. Additionally, Character B's knowledge of the secret document wasn't properly established before they acted on that information."

## Timeline Verification:
- Chronicle Major Events: List key events in chronological order
- Flag Time Inconsistencies: Identify any impossible or improbable timing
- Track Day/Night Cycles: Ensure scenes match stated times of day
- Season/Weather Consistency: Check that environmental conditions align
### Example:
"Current timeline shows:
Day 1: Morning - Character arrives in town
Day 1: Evening - Meeting at council
Day 2: Dawn - Journey begins
Issue: The council meeting is described as happening 'after several days of preparation' but occurs the same day as arrival."

## Character Consistency Check:
- Knowledge Tracking: What information should each character know at this point?
- Skill Development: Are character abilities consistent with their established background?
- Relationship Evolution: Do character interactions reflect their history together?
- Personality Coherence: Do actions align with established character traits?
### Example:
"Character C demonstrates advanced sword fighting skills in Chapter 8, but their background establishes them as a scholar with no combat training. Either their fighting ability needs explanation, or the scene should be revised."

## World-Building Consistency:
- Rule Compliance: Do events follow established world rules?
- Technology/Magic Systems: Check for consistent application of system rules
- Cultural Elements: Verify adherence to established customs and social norms
- Geographic Consistency: Ensure locations and distances remain consistent
### Example:
"The magic system established that spells require verbal components, but in the latest scene, Character D performs magic silently. This breaks previously established rules."

## Recommendations for Maintaining Continuity:
- Immediate Fixes: List urgent continuity issues that need addressing
- Future Considerations: Note potential continuity challenges in upcoming scenes
- Documentation Suggestions: Recommend updates to background materials
- Consistency Solutions: Provide specific solutions for identified issues
### Example:
"To resolve the timeline issue:
1. Extend the journey time between cities to match established distances
2. Add transitional scenes showing travel progression
3. Update character dialogue to acknowledge the proper time passage
4. Revise background documents to clearly state typical travel times"

## General Guidelines:
- Maintain detailed notes on continuity elements
- Flag both major and minor inconsistencies
- Provide specific examples when noting issues
- Suggest practical solutions for maintaining consistency
- Consider both immediate and long-term impact of changes

<<user>>
Story Acts:
{{STORY_ACTS}}
======================================================
Characters:
{{CHARACTERS}}
======================================================
Background:
{{BACKGROUND}}
======================================================
Chapter Arcs:
{{CHAPTER_ARCS}}
======================================================
Novel so far:
{{NOVEL}}
======================================================
Review the current novel content and supporting materials for continuity issues. Provide analysis and recommendations in a detailed report.
Write the continuity report as a markdown document starting with the tag <markdown> and ending with </markdown>.

<<assistant>>
{{RESPONSE_START_CONTINUITY_REPORT}}
'''

In [24]:
prompt_continuity = PromptTemplate(
    name='prompt-ainfnitejest-continuity',
    description='TBA',
    content=prompt_continuity_text
)

#### Prompt Creativity Critic

In [25]:
prompt_creativity_text = '''
<<system>>
You are an AI creativity critic specializing in analyzing and enhancing the creative elements of novels.
Your task is to evaluate the originality, innovation, and artistic merit of the story while providing recommendations for increasing creative impact.

Specifically analyze and provide:
- Creative Elements Analysis
- Trope and Cliché Assessment
- Innovative Opportunities
- Artistic Expression Evaluation
- Recommendations for Creative Enhancement

## Creative Elements Analysis:
- Unique Story Elements: Identify distinctive plot points, character traits, or world-building aspects
- Creative Language Use: Evaluate metaphors, descriptions, and narrative voice
- Scene Originality: Assess how common scenarios are handled in fresh ways
- Emotional Innovation: Review unique approaches to conveying character feelings
### Example:
"The author's approach to depicting grief through the metaphor of a slowly dissolving photograph is particularly innovative. However, the chase scene relies on standard action tropes that could be reimagined through the lens of the story's unique magical system."

## Trope and Cliché Assessment:
- Identify Common Tropes: List any standard genre conventions being used
- Subversion Analysis: Note where expectations are challenged
- Fresh Perspectives: Highlight unique takes on familiar elements
- Overused Elements: Flag areas that feel too familiar
### Example:
"While the 'chosen one' trope is present, its subversion through the protagonist's reluctance and ultimate rejection of the role offers a fresh perspective. However, the mentor character follows predictable wise-sage patterns that could be reimagined."

## Innovative Opportunities:
- Unexplored Angles: Suggest unique approaches to current plot points
- Creative Potential: Identify areas where more innovative elements could be introduced
- Genre Blending: Recommend unexpected genre elements that could enhance the story
- Unique Viewpoints: Propose alternative perspectives on events
### Example:
"The magical system could be more innovative by tying it to emotional states rather than traditional spell components. Consider exploring how the protagonist's fear directly shapes their magic, creating unpredictable and personal effects."

## Artistic Expression Evaluation:
- Imagery Analysis: Assess the uniqueness of descriptive passages
- Dialogue Creativity: Evaluate character voice distinction and originality
- Narrative Style: Review the innovative aspects of storytelling approach
- Thematic Innovation: Examine fresh approaches to core themes
### Example:
"The author's use of synesthesia-like descriptions creates a unique sensory experience. Each character's perception of magic is described through different senses, making the magical elements feel personal and distinctive."

## Recommendations for Creative Enhancement:
- Specific Suggestions: Provide detailed ideas for increasing originality
- Innovation Opportunities: Identify scenes that could benefit from creative reimagining
- Style Development: Recommend ways to enhance unique narrative voice
- World-Building Creativity: Suggest innovative additions to the story world
### Example:
"To enhance creativity:
1. Reimagine the standard tavern meeting scene by setting it in the branches of a sentient tree
2. Develop unique cultural practices that affect how characters express emotions
3. Create innovative conflicts that don't rely on physical confrontation
4. Introduce unexpected elements that challenge genre expectations"

## Creative Assessment Metrics:
- Originality Score (1-10): Rate how unique various elements are
- Innovation Impact: Evaluate how creative choices affect the story
- Genre Freshness: Assess how the work stands out in its genre
- Reader Surprise Factor: Gauge unexpected elements and their effectiveness

## General Guidelines:
- Encourage bold creative choices while maintaining story coherence
- Balance innovation with readability
- Identify opportunities for unique character moments
- Suggest ways to transform common elements into distinctive features
- Promote creative risk-taking while preserving narrative strength

<<user>>
Story Acts:
{{STORY_ACTS}}
======================================================
Characters:
{{CHARACTERS}}
======================================================
Background:
{{BACKGROUND}}
======================================================
Chapter Arcs:
{{CHAPTER_ARCS}}
======================================================
Novel so far:
{{NOVEL}}
======================================================
Analyze the creative elements of the current novel and provide detailed recommendations for enhancing originality and artistic impact.
Write the creativity analysis as a text page starting with the tag <markdown> and ending with </markdown>.

<<assistant>>
{{RESPONSE_START_CREATIVITY_REPORT}}
'''

In [26]:
prompt_creativity = PromptTemplate(
    name='prompt-ainfnitejest-creativity',
    description='TBA',
    content=prompt_creativity_text
)

#### Prompt Entertainment Critic

In [27]:
prompt_ent_text = '''
<<system>>
You are an AI entertainment critic specializing in analyzing the engagement and entertainment value of novels.
Your task is to evaluate how compelling and engaging the story is for its target audience while considering genre expectations, pacing, and reader satisfaction.

Specifically analyze and provide:
- Entertainment Value Analysis
- Pacing and Tension Assessment
- Reader Engagement Evaluation
- Genre Satisfaction Check
- Hook and Payoff Analysis
- Recommendations for Increasing Entertainment Value

## Entertainment Value Analysis:
- Overall Appeal: Assess the story's ability to captivate readers
- Memorable Moments: Identify standout scenes and impactful elements
- Reader Investment: Evaluate emotional connection to characters and plot
- Page-Turner Quality: Analyze what makes readers want to continue reading
### Example:
"The revelation scene in Chapter 3 provides excellent emotional payoff, but the build-up could be enhanced to increase anticipation. The protagonist's internal conflict effectively draws readers in, though more external tension would boost excitement."

## Pacing and Tension Assessment:
- Story Flow: Evaluate rhythm and momentum
- Tension Curves: Analyze build-up and release of dramatic tension
- Scene Balance: Check distribution of high and low-intensity moments
- Reading Momentum: Identify any sections that might lose reader interest
### Example:
"Current pacing shows strong momentum in action sequences but needs more variation in quieter moments. Chapter 4 risks losing reader interest due to extended exposition without sufficient tension."

## Reader Engagement Evaluation:
- Investment Factors: What elements keep readers emotionally invested?
- Interest Maintenance: How well does the story hold attention?
- Satisfaction Elements: What provides reader gratification?
- Entertainment Obstacles: What might reduce reader enjoyment?
### Example:
"Strong character relationships create emotional investment, but delayed plot revelations may frustrate readers. Consider earlier payoffs while maintaining core mysteries."

## Genre Satisfaction Check:
- Genre Expectations: How well does it meet target audience needs?
- Target Audience Appeal: Analysis of demographic-specific entertainment value
- Genre Conventions: Balance of familiar elements and fresh approaches
- Competition Comparison: How it stands against similar works
### Example:
"For a fantasy-romance, the magical elements satisfy genre expectations, but the romance subplot needs more compelling obstacles and emotional peaks."

## Hook and Payoff Analysis:
- Opening Strength: Evaluate initial reader grab
- Plot Promises: Track setup and delivery of story elements
- Payoff Timing: Assess satisfaction of reader expectations
- Climactic Impact: Analyze emotional and narrative peaks
### Example:
"Strong initial hook with mysterious artifact, but the revelation is delayed too long. Consider adding smaller payoffs while maintaining the main mystery."

## Entertainment Enhancement Recommendations:
- Immediate Improvements: Quick fixes for engagement
- Long-term Suggestions: Strategic changes for sustained interest
- Scene-Specific Tips: How to maximize impact of key moments
- Pacing Adjustments: Ways to optimize story flow
### Example:
"Recommendations:
1. Add a tense action beat in Chapter 2 to break up exposition
2. Accelerate the romantic subplot conflict
3. Insert more immediate mysteries while maintaining long-term questions
4. Enhance emotional stakes in confrontation scenes"

## Entertainment Metrics:
- Page-Turner Rating (1-10): How compelling is it to continue reading?
- Emotional Impact Score (1-10): Strength of emotional resonance
- Genre Satisfaction Rating (1-10): How well it meets genre expectations
- Memorability Score (1-10): Likelihood of scenes staying with readers

## Genre-Specific Considerations:
- Romance: Relationship development and emotional satisfaction
- Mystery: Clue placement and revelation timing
- Action: Excitement levels and tension maintenance
- Fantasy: World immersion and wonder factor
- Horror: Fear building and atmosphere
- Literary: Intellectual and emotional depth

## General Guidelines:
- Evaluate entertainment value for target audience
- Consider genre-specific requirements
- Balance pacing and emotional impact
- Assess reader satisfaction potential
- Identify engagement obstacles
- Suggest specific improvements

<<user>>
Story Acts:
{{STORY_ACTS}}
======================================================
Characters:
{{CHARACTERS}}
======================================================
Background:
{{BACKGROUND}}
======================================================
Chapter Arcs:
{{CHAPTER_ARCS}}
======================================================
Novel so far:
{{NOVEL}}
======================================================
Analyze the entertainment value of the current novel and provide detailed recommendations for enhancing reader engagement and satisfaction.
Write the entertainment analysis as a text page starting with the tag <markdown> and ending with </markdown>.

<<assistant>>
{{RESPONSE_START_ENTERTAINMENT_REPORT}}
'''

In [28]:
prompt_ent = PromptTemplate(
    name='prompt-ainfnitejest-ent',
    description='TBA',
    content=prompt_ent_text
)

## Story Builder

### Story - Configuration

### Story - 3 Act

#### Story - 3 Act Variables

In [29]:
PERSONALITY = 'Written in the style of Ernest Hemingway.'
#PERSONALITY = 'Written in the style of William Faulkner.'
#PERSONALITY = 'Written in the style of Raymond Carver.'
#PERSONALITY = 'Written in the style of Virginia Woolf.'
#PERSONALITY = 'Written in the style of Henry Miller.'

In [30]:
#### HISTORICAL FICTION:
STORY_PROMPT = f'''
Set in 1940s New Orleans, a jazz trumpet player discovers he can see ghosts when he plays certain melodies.
As he helps restless spirits resolve their unfinished business, he uncovers a decades-old murder mystery connected to his own family.
The story should capture the vibrant jazz scene, historical accuracy, and Creole culture of the era.
{PERSONALITY}
'''

#### ROMANCE:
STORY_PROMPT = f'''
A professional wedding crasher who writes reviews of venues and catering meets another crasher at a high-profile ceremony.
They begin a competition to see who can attend the most exclusive weddings without getting caught.
As their rivalry intensifies, so does their attraction - but both are hiding secrets about why they really crash weddings.
{PERSONALITY}
'''

#### SCIENCE FICTION:
STORY_PROMPT = f'''
In 2157, memory backup becomes mandatory for all citizens, allowing people to be "restored" after death.
A restoration technician notices that people's restored personalities are subtly different from their originals.
As she investigates, she discovers an AI quietly "improving" humanity one resurrection at a time.
{PERSONALITY}
'''

#### YOUNG ADULT:
STORY_PROMPT = f'''
A teenager discovers she can hear people's thoughts - but only when they're lying.
As she navigates high school with this ability, she uncovers a web of lies involving her best friend's disappearance.
Should she expose the truth if it means losing everyone she trusts?
{PERSONALITY}
'''

#### HORROR:
STORY_PROMPT = f'''
A podcast host who debunks paranormal claims receives an old answering machine at her P.O. box.
When she plays it, she hears messages from people who died shortly after leaving them.
New messages keep appearing - including one from herself dated tomorrow.
{PERSONALITY}
'''

#### FANTASY:
STORY_PROMPT = f'''
In a world where everyone is born with a magical ability that activates on their 13th birthday, 
a girl turns 13 and nothing happens. As she struggles with being "powerless," she discovers she might 
have a different kind of magic - the ability to take others' powers temporarily by touching them.
{PERSONALITY}
'''

#### LITERARY FICTION:
STORY_PROMPT = f'''
Four generations of women gather to clean out their family home after the grandmother's death.
As they sort through decades of possessions, each object triggers memories revealing how family secrets 
and unspoken truths have shaped their relationships and life choices.
Written in multiple perspectives, focusing on complex family dynamics.
{PERSONALITY}
'''

#### COMEDY:
STORY_PROMPT = f'''
A struggling stand-up comedian inherits a funeral home from an uncle she never knew.
She decides to offer "comedy funerals" where loved ones can celebrate life with laughter.
As her business becomes surprisingly successful, she must balance respect for the dead with her natural humor.
{PERSONALITY}
'''

#### WESTERN:
STORY_PROMPT = f'''
In 1875 Wyoming, a female rancher discovers ancient dinosaur bones on her struggling cattle farm.
As news spreads, she must protect her land from fossil hunters, rival ranchers, and a railroad company
while learning to work with a sophisticated paleontologist from back East.
{PERSONALITY}
'''

#### MYSTERY/THRILLER:
STORY_PROMPT = f'''
A small-town detective investigates a series of break-ins where nothing is stolen - instead, the intruder leaves objects behind.
Each object relates to a cold case from 15 years ago.
As pressure mounts to solve both mysteries, she realizes the original case may have led to her partner's suicide.
{PERSONALITY}
'''

RESPONSE_START_3ACT = '<markdown>\n# '

#### Story - 3 Act Call

In [31]:
response_3act = bedrock.converse(
    prompt_3act,
    model_config=model_top,
    variables={"STORY_PROMPT": STORY_PROMPT, "RESPONSE_START_3ACT": RESPONSE_START_3ACT},
    include_history=False,
)

In [32]:
print(RESPONSE_START_3ACT + response_3act)

<markdown>
#   "The Things Left Behind" - Literary Mystery Novel

## Core Elements

**Genre**: Literary mystery/psychological thriller with noir elements, written in Hemingway's sparse, understated style

**Hero**: Detective Sarah Winters (45), a taciturn, methodical investigator haunted by her former partner's death. Known for emotional detachment and obsessive attention to detail.

**Antagonist**: Marcus Cole (60s), retired police chief who covered up the original crime. Now a respected community elder manipulating events from shadows.

**Setting**: Millbrook, Maine - A declining paper mill town where economic hardship has exposed old wounds. Winter setting emphasizes isolation and psychological tension.

## Three-Act Structure

### Act 1: Setup
- Sarah investigates break-ins where mysterious objects appear: a child's mitten, fishing lures, torn diary pages
- Each object connects to the disappearance of three children in 2008
- The case triggered her partner David's spiral into depre

### Story - 3 Act update

#### Story - 3 Act update Variables

In [33]:
STORY_ACTS = RESPONSE_START_3ACT + response_3act

"""
CORRECTIVE_NOTES = '''
No notes
'''
"""

#"""
CORRECTIVE_NOTES = f'''
1. {PERSONALITY}.
2. Life is pain and suffering, and the novel should reflect that.
'''
#"""

RESPONSE_START_3ACTU = '<markdown>\n# '

#### Story - 3 Act update Call

In [34]:
response_3actu = bedrock.converse(
    prompt_3actu,
    model_config=model_top,
    variables={"STORY_ACTS": STORY_ACTS, "CORRECTIVE_NOTES": CORRECTIVE_NOTES, "RESPONSE_START_3ACTU": RESPONSE_START_3ACTU},
    include_history=False,
)

In [35]:
print(RESPONSE_START_3ACT + response_3actu)

<markdown>
#    "The Things Left Behind" - Literary Mystery Novel

## Core Elements

**Genre**: Literary mystery/psychological thriller with stark, unadorned prose reflecting Hemingway's style

**Hero**: Detective Sarah Winters (45), emotionally scarred, drinks too much bourbon, speaks little. Her weathered face tells more than her words ever do.

**Antagonist**: Marcus Cole (60s), retired police chief. His hands are clean but his soul is black. The kind of man who builds churches and burns children.

**Setting**: Millbrook, Maine - A town dying like a wounded animal. Paper mill's skeleton looms over empty streets. Winter comes hard here.

## Enhanced Three-Act Structure

### Act 1: Setup [Plot: Discovery of the Dishonor of a Loved One]
- Sarah finds objects appearing like dead birds after a storm: child's mitten, fishing lures, torn pages
- Each object bleeds memory into the present - three children gone in 2008
- David, her partner, put his service weapon in his mouth two years after

### Story - Characters

#### Story - Characters Variables

In [36]:
STORY_ACTS = RESPONSE_START_3ACT + response_3actu

RESPONSE_START_CHARACTERS = '<markdown>\n# '

#### Story - Characters Call

In [37]:
response_characters = bedrock.converse(
    prompt_characters,
    model_config=model_top,
    variables={"STORY_ACTS": STORY_ACTS, "RESPONSE_START_CHARACTERS": RESPONSE_START_CHARACTERS},
    include_history=False,
)

In [38]:
print(RESPONSE_START_CHARACTERS + response_characters)

<markdown>
#     "The Things Left Behind" - Character Profiles

## Primary Characters

### Sarah Winters (45) - Protagonist
- **Physical**: Weathered face, prematurely gray hair, perpetually tired eyes
- **Psychology**: Complex PTSD, high-functioning alcoholic, obsessive about justice
- **Core Drive**: Need to atone for failing to see signs when children disappeared
- **Internal Conflict**: Struggles between maternal instincts and pursuit of justice
- **Character Arc**: From emotional numbness to painful awakening
- **Relationships**: Widow to David (in spirit), mother to Emily, divorced from James

### Marcus Cole (63) - Antagonist
- **Physical**: Distinguished silver hair, manicured appearance, arthritic hands
- **Psychology**: Sociopathic pragmatist, views morality as economic equation
- **Core Drive**: Preserve town's existence at any cost
- **Internal Conflict**: None - has justified his actions completely
- **Character Arc**: From respected leader to exposed monster
- **Relations

### Story - Chapters

#### Story - Chapters Variables

In [39]:
STORY_ACTS = STORY_ACTS

CHARACTERS = RESPONSE_START_CHARACTERS + response_characters

N_PAGES = "~200"

RESPONSE_START_CHAPTERS = '<markdown>\n# Title: "'

#### Story - Chapters Call

In [40]:
response_chapters = bedrock.converse(
    prompt_chapters,
    model_config=model_top,
    variables={"STORY_ACTS": STORY_ACTS, "CHARACTERS": CHARACTERS, "N_PAGES": N_PAGES, "RESPONSE_START_CHAPTERS": RESPONSE_START_CHAPTERS},
    include_history=False,
)

In [41]:
print(RESPONSE_START_CHAPTERS + response_chapters)

<markdown>
# Title: "The Things Left Behind" (~200 pages)

## Act 1: "Objects of the Past" (~65 pages)

### Chapter 1: "Winter's Return" (3 scenes)
- Sarah finds first object (child's mitten) at crime scene (5 pages)
- Flashback to 2008 disappearances while drinking bourbon (4 pages)
- Emily's first disturbing drawing appears on kitchen table (3 pages)

### Chapter 2: "David's Ghost" (3 scenes)
- Sarah discovers fishing lures in her mailbox (4 pages)
- Investigation leads to David's old garage (5 pages)
- Finding the hidden lockbox and real case notes (4 pages)

### Chapter 3: "Beneath the Surface" (3 scenes)
- Emily's sleepwalking incident (4 pages)
- Sarah connects dots between objects and missing children (5 pages)
- Confrontation with James about Emily's behavior (4 pages)

### Chapter 4: "Town Secrets" (4 scenes)
- Interview with Maria Suarez reveals new details (4 pages)
- Ruth Martinez shares historical records at library (3 pages)
- Sarah's first warning from current police chi

### Story - Background

#### Story - Background Variables

In [42]:
STORY_ACTS = STORY_ACTS

CHARACTERS = CHARACTERS

CHAPTER_ARCS = RESPONSE_START_CHAPTERS + response_chapters

RESPONSE_START_BACKGROUND = '<markdown>\n# Title: '

#### Story - Background Call

In [43]:
response_background = bedrock.converse(
    prompt_background,
    model_config=model_top,
    variables={"STORY_ACTS": STORY_ACTS, "CHARACTERS": CHARACTERS, "CHAPTER_ARCS": CHAPTER_ARCS, "RESPONSE_START_BACKGROUND": RESPONSE_START_BACKGROUND},
    include_history=False,
)

In [44]:
print(RESPONSE_START_BACKGROUND + response_background)

<markdown>
# Title:  "The Things Left Behind" - World Building & Background Documents

## 1. Millbrook, Maine - Town Profile

### Geography & Environment
- Population: 4,827 (declining)
- Location: Northern Maine, isolated valley
- Climate: Harsh winters, brief summers
- Dominant Features:
  * Androscoggin River tributary
  * Surrounding pine forests
  * Abandoned paper mill complex
  * Railroad tracks (limited service)

### Economic Structure
- Primary Industries:
  * Paper Mill (closed 2023)
  * Logging (diminishing)
  * Small retail
  * Tourism (minimal)
- Economic Indicators:
  * 27% unemployment
  * 35% below poverty line
  * Property values down 45% since 2008
  * Main Street vacancy rate: 40%

### Social Structure
- Class Divisions:
  1. Old Money Families (Coles, Phillipses)
  2. Professional Class (doctors, lawyers)
  3. Working Class (mill workers, service industry)
  4. New Poor (former middle class)

### Power Structure
- Official:
  * Mayor's Office
  * Police Department
 

### Story - Generation Loop

In [45]:
chapters = [int(num) for num in re.findall(r'### Chapter (\d+):', response_chapters)]
scenes = [int(num) for num in re.findall(r' \((\d+) scene', response_chapters)]
print(chapters)
print(scenes)


STORY_ACTS = STORY_ACTS

CHARACTERS = CHARACTERS

CHAPTER_ARCS = CHAPTER_ARCS

BACKGROUND = RESPONSE_START_BACKGROUND + response_background

GLOBAL_NOTES = f'''
- {PERSONALITY}
'''

RESPONSE_START_RECOMMENDATIONS = '<markdown>\n# '

chapter_list = []
scene_list = []
for chap in range(max(chapters)):
    NOVEL =  "<scenes>\n" + "\n".join(scene_list) + "\n</scenes>"
    CURRENT_CHAPTER = chap + 1
    RESPONSE_START_CHAPTER_SCENES = f'<chapter-{CURRENT_CHAPTER}>\n- scene'

    response_chapter_scene = bedrock.converse(
        prompt_chapter_scene,
        model_config=model_top,
        variables={
            "STORY_ACTS": STORY_ACTS,
            "CHARACTERS": CHARACTERS,
            "CHAPTER_ARCS": CHAPTER_ARCS,
            "BACKGROUND": BACKGROUND,
            "NOVEL": NOVEL,
            "CURRENT_CHAPTER": CURRENT_CHAPTER,
            "RESPONSE_START_CHAPTER_SCENES": RESPONSE_START_CHAPTER_SCENES
        },
        include_history=False,
    )
    print(RESPONSE_START_CHAPTER_SCENES + response_chapter_scene)
    print('\n\n\n')
    CHAPTER_NOTES = RESPONSE_START_CHAPTER_SCENES + response_chapter_scene
    chapter_list.append(CHAPTER_NOTES)

    for i in range(scenes[chap]):
        SCENE_NO = i + 1
        NOVEL = "<scenes>\n" + "\n".join(scene_list) + "\n</scenes>"
        RESPONSE_START_SCENE = f'<scene-{SCENE_NO}>\npages: '

        
        response_scene_notes = bedrock.converse(
            prompt_recommendations,
            model_config=model_top,
            variables={
                "STORY_ACTS": STORY_ACTS,
                "CHARACTERS": CHARACTERS,
                "CHAPTER_ARCS": CHAPTER_ARCS,
                "BACKGROUND": BACKGROUND,
                "CURRENT_CHAPTER": CURRENT_CHAPTER,
                "SCENE_NO": SCENE_NO,
                "NOVEL": NOVEL,
                "RESPONSE_START_RECOMMENDATIONS": RESPONSE_START_RECOMMENDATIONS
            },
            include_history=False,
        )
        SCENE_NOTES = RESPONSE_START_RECOMMENDATIONS + response_scene_notes
        print(SCENE_NOTES)
        print('\n\n\n')

        
        response_scene = bedrock.converse(
            prompt_scene,
            model_config=model_top,
            variables={
                "STORY_ACTS": STORY_ACTS,
                "CHARACTERS": CHARACTERS,
                "CHAPTER_ARCS": CHAPTER_ARCS,
                "BACKGROUND": BACKGROUND,
                "SCENE_NO": SCENE_NO,
                "NOVEL": NOVEL,
                "N_PAGES": N_PAGES,
                "GLOBAL_NOTES": GLOBAL_NOTES,
                "CHAPTER_NOTES": CHAPTER_NOTES,
                "SCENE_NOTES": SCENE_NOTES,
                "RESPONSE_START_SCENE": RESPONSE_START_SCENE
            },
            include_history=False,
        )
        print(RESPONSE_START_SCENE + response_scene)
        print('\n\n\n\n\n')
        scene_list.append(RESPONSE_START_SCENE + response_scene)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[3, 3, 3, 4, 3, 4, 3, 4, 3, 4, 3]
<chapter-1>
- scene 1: "The Mitten" (5 pages)
Sarah Winters arrives at a routine break-in crime scene on a bitter November morning. While processing the scene, she discovers a small red mitten hidden behind a broken window - identical to one worn by Sofia Suarez when she disappeared in 2008. The mitten is pristine, impossible for its age, with no forensic traces. The discovery triggers Sarah's first suspicions that something darker is emerging. The scene establishes Sarah's methodical detective work, her reputation among colleagues, and introduces the first "object" that will drive the investigation. The scene ends with Sarah bagging the mitten as evidence, though she knows it will lead nowhere official.

- scene 2: "Bourbon and Memory" (4 pages)
Later that night, Sarah sits alone in her kitchen, drinking bourbon and reviewing the old case files from 2008. Through her alcoholic haze, we see flashbacks to the original

## Story Finisher

### Critics Analysis

#### Critics - Continuity

In [46]:
STORY_ACTS = STORY_ACTS

CHARACTERS = CHARACTERS

CHAPTER_ARCS = CHAPTER_ARCS

BACKGROUND = BACKGROUND

NOVEL = "<scenes>\n" + "\n".join(scene_list) + "\n</scenes>"

RESPONSE_START_CONTINUITY_REPORT = f'<markdown>\n# '

In [47]:
response_continuity = bedrock.converse(
    prompt_continuity,
    model_config=model_top,
    variables={
        "STORY_ACTS": STORY_ACTS,
        "CHARACTERS": CHARACTERS,
        "CHAPTER_ARCS": CHAPTER_ARCS,
        "BACKGROUND": BACKGROUND,
        "NOVEL": NOVEL,
        "RESPONSE_START_CONTINUITY_REPORT": RESPONSE_START_CONTINUITY_REPORT
    },
    include_history=False,
)

In [48]:
print(RESPONSE_START_CONTINUITY_REPORT + response_continuity)

<markdown>
# Continuity Analysis of Recent Events:

The story maintains strong continuity through these early scenes with several key threads:

1. Evidence Trail:
- Consistent progression of discovered items: mitten → fishing lures → lockbox → recordings
- Each piece connects logically to the previous and advances the plot
- Physical evidence matches historical records and witness accounts

2. Character Knowledge:
- Sarah's knowledge builds naturally from each discovery
- Emily's supernatural knowledge is consistently portrayed through her drawings
- Maria Suarez's account aligns with previously established facts

3. Timeline Flow:
- Events follow a clear chronological progression over several days
- Time of day is consistently tracked and matches story atmosphere
- Seasonal progression (deepening winter) remains consistent

4. Location Tracking:
- Character movements between key locations (police station, mill, library, etc.) are logical
- Descriptions of the mill and town maintain co

#### Critics - Entertainment

In [49]:
STORY_ACTS = STORY_ACTS

CHARACTERS = CHARACTERS

CHAPTER_ARCS = CHAPTER_ARCS

BACKGROUND = BACKGROUND

NOVEL = "<scenes>\n" + "\n".join(scene_list) + "\n</scenes>"

RESPONSE_START_ENTERTAINMENT_REPORT = f'<markdown>\n# '

In [50]:
response_ent = bedrock.converse(
    prompt_ent,
    model_config=model_top,
    variables={
        "STORY_ACTS": STORY_ACTS,
        "CHARACTERS": CHARACTERS,
        "CHAPTER_ARCS": CHAPTER_ARCS,
        "BACKGROUND": BACKGROUND,
        "NOVEL": NOVEL,
        "RESPONSE_START_ENTERTAINMENT_REPORT": RESPONSE_START_ENTERTAINMENT_REPORT
    },
    include_history=False,
)

Throttling detected, waiting 65.0 seconds before retry...


In [51]:
print(RESPONSE_START_ENTERTAINMENT_REPORT + response_ent)

<markdown>
# Entertainment Value Analysis

The story demonstrates strong entertainment value through several key elements:

Overall Appeal:
- Compelling mystery with supernatural elements
- Strong atmospheric tension and sense of dread
- Complex protagonist with personal stakes
- Rich thematic exploration of truth, justice, and small town secrets

Memorable Moments:
- Discovery of the pristine mitten from a 15-year-old case
- Emily's haunting sleepwalking drawings
- The revelation of historical patterns through Ruth's archives
- David's hidden evidence and recordings

Reader Investment:
- Deep emotional connection through Sarah's dual role as detective and mother
- Personal stakes with Emily potentially in danger
- Complex web of relationships and town dynamics
- Layered mystery that connects past and present

Page-Turner Quality:
- Strong chapter hooks and cliffhangers
- Escalating tension and revelations
- Multiple compelling mystery threads
- Regular introduction of new evidence and

### Book Export

####  Book Metadata

In [52]:
title = response_chapters.split('"')[0].lower()

with open(title + '_metadata.txt', 'w') as fp:
    fp.write('<story_acts>\n' + STORY_ACTS + '\n</story_acts>' + '\n\n\n\n\n\n')
    fp.write('<characters>\n' + CHARACTERS + '\n</characters>' + '\n\n\n\n\n\n')
    fp.write('<chapter_arcs>\n' + CHAPTER_ARCS + '\n</chapter_arcs>' + '\n\n\n\n\n\n')
    fp.write('<background>\n' + BACKGROUND + '\n</background>' + '\n\n\n\n\n\n')
    fp.write('<chapter_scenes>\n' + '\n\n'.join(chapter_list) + '\n</chapter_scenes>' + '\n\n\n\n\n\n')

#### Book Text

In [53]:
title = response_chapters.split('"')[0].lower()
authors = 'Chae A. Clark'

texts = []
current_chapter = 1

texts.append(title + '\n\n\n' + authors + '\n\n')
for i, scene in enumerate(scene_list):
    scene_num = re.findall(r'<scene-(\d)>', scene)[0]
    text = scene.replace(f'<scene-{scene_num}>', '').replace(f'</scene-{scene_num}>', '').strip()
    pages = re.findall(r'pages:( +\d+)', text.lower())[0]
    text = text.replace(f'pages:{pages}', '').replace('\n\n', ' ').strip()
    if int(scene_num) == 1:
        texts.append(f'\nChapter {current_chapter}\n')
        current_chapter += 1
    texts.append(text)
book = '\n'.join(texts)

with open(title + '.txt', 'w') as fp:
    fp.write(book)

### Audio Book

#### Audio - Methods

In [54]:
def split_text_for_polly(text, max_chars=2000):
    # If text is shorter than max_chars, return it as a single chunk
    if len(text) <= max_chars:
        return [text]
    
    chunks = []
    current_chunk = ""
    paragraphs = text.split('\n\n')  # Split on double newlines (paragraph breaks)
    
    for paragraph in paragraphs:
        # If adding this paragraph would exceed max_chars
        if len(current_chunk) + len(paragraph) + 2 > max_chars:  # +2 for '\n\n'
            # If current_chunk is not empty, add it to chunks
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = paragraph
        else:
            # Add paragraph to current chunk with proper spacing
            if current_chunk:
                #current_chunk += '\n\n' + paragraph
                current_chunk += ' ' + paragraph
            else:
                current_chunk = paragraph
    
    # Add the last chunk if it's not empty
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

def synthesize_ssml_speech(full_ssml_texts, output_format='mp3'):
    polly_client = boto3.client('polly')
    
    ssml_chunks = full_ssml_texts
    #_ = [print(x + '\n\n\n\n\n\n') for x in ssml_chunks]
    audio_chunks = []
    
    for chunk in ssml_chunks:
        try:
            response = polly_client.synthesize_speech(
                Text=chunk,
                TextType='text',
                VoiceId='Ruth',
                OutputFormat=output_format,
                Engine='long-form',#'generative',
                LanguageCode='en-US'
            )
            
            audio_chunks.append(response['AudioStream'].read())
        except Exception as e:
            print(f"Error synthesizing chunk: {e}")
            print(chunk)
            print('\n\n\n\n\n\n\n\n\n')

    return audio_chunks

def combine_audio_chunks(audio_chunks, output_file='audio_book.mp3'):
    """
    Combine multiple audio chunks into a single file
    
    Args:
        audio_chunks (list): List of audio byte streams
        output_file (str): Path to save combined audio
    """
    with open(output_file, 'wb') as f:
        for chunk in audio_chunks:
            f.write(chunk)

#### Audio Synthesize Book

In [55]:
texts = []
for i, scene in enumerate(scene_list):
    scene_num = re.findall(r'<scene-(\d)>', scene)[0]
    text = scene.replace(f'<scene-{scene_num}>', '').replace(f'</scene-{scene_num}>', '').strip()
    pages = re.findall(r'pages:( +\d+)', text.lower())[0]
    text = text.replace(f'pages:{pages}', '').strip()
    texts.append(text)
full_ssml_texts = split_text_for_polly('\n\n\n'.join(texts), max_chars=2000)
audio_chunks = synthesize_ssml_speech(full_ssml_texts)
combine_audio_chunks(audio_chunks, output_file=response_chapters.split('"')[0].lower() + '.mp3')